In [2]:
pip install torch torchvision

  Obtaining dependency information for torch from https://files.pythonhosted.org/packages/18/cf/f69dff972a748e08e1bf602ef94ea5c6d4dd2f41cea22c8ad67a607d8b41/torch-2.4.0-cp311-cp311-win_amd64.whl.metadata
  Using cached torch-2.4.0-cp311-cp311-win_amd64.whl.metadata (27 kB)
  Obtaining dependency information for torchvision from https://files.pythonhosted.org/packages/6a/38/e8257ad99ea2ec30bbb4e6d9d81f3fe796e39046bb060d80569cbf6d83e5/torchvision-0.19.0-1-cp311-cp311-win_amd64.whl.metadata
  Using cached torchvision-0.19.0-1-cp311-cp311-win_amd64.whl.metadata (6.1 kB)
Using cached torch-2.4.0-cp311-cp311-win_amd64.whl (197.9 MB)
Using cached torchvision-0.19.0-1-cp311-cp311-win_amd64.whl (1.3 MB)
Note: you may need to restart the kernel to use updated packages.


In [6]:
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

# 이미지 변환 설정
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # VGG 모델의 입력 크기
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# 데이터셋 로드 (경로 수정)
train_dataset = ImageFolder(root='C://Users//peter//Desktop//pet//master//train', transform=transform)
val_dataset = ImageFolder(root='C://Users//peter//Desktop//pet//master//valid', transform=transform)

# 데이터로더 설정
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# 데이터셋 클래스 확인
print(train_dataset.classes)


['Angry', 'Other', 'Sad', 'happy']


In [ ]:
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
import torchvision
from tqdm import tqdm  # 진행 바 라이브러리

# 이미지 변환 설정
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # VGG 모델의 입력 크기
    transforms.ToTensor(),  # 이미지를 텐서로 변환
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # 정규화
])

# 데이터셋 로드 (경로 수정)
train_dataset = ImageFolder(root='C://Users//peter//Desktop//pet//master//train', transform=transform)
val_dataset = ImageFolder(root='C://Users//peter//Desktop//pet//master//valid', transform=transform)

# 데이터로더 설정
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# GPU 사용 여부 확인
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

# VGG 모델 로드 및 수정
model = torchvision.models.vgg16(pretrained=True)
model.classifier[6] = nn.Linear(4096, 4)  # 4개의 클래스 분류
model = model.to(device)  # 모델을 GPU/CPU로 이동

# 손실 함수 및 최적화기 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 모델 학습 함수 (GPU 사용 추가)
def train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        # tqdm을 사용하여 진행 바 추가
        train_loader_tqdm = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}', leave=False)
        for inputs, labels in train_loader_tqdm:
            inputs, labels = inputs.to(device), labels.to(device)  # 데이터를 GPU/CPU로 이동

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

            # 진행 바 설명 업데이트
            train_loader_tqdm.set_postfix(loss=running_loss/len(train_loader))

        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

        # 평가
        evaluate_model(model, val_loader)

# 모델 평가 함수 (GPU 사용 추가)
def evaluate_model(model, val_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        val_loader_tqdm = tqdm(val_loader, desc='Validation', leave=False)
        for inputs, labels in val_loader_tqdm:
            inputs, labels = inputs.to(device), labels.to(device)  # 데이터를 GPU/CPU로 이동
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Validation Accuracy: {100 * correct / total:.2f}%')

# 모델 학습
train_model(model, criterion, optimizer, train_loader, val_loader)

# 모델 저장
torch.save(model.state_dict(), 'pet_expression_model.pth')


Using device: cpu


Epoch [1/10], Loss: 1.4847


Validation Accuracy: 25.00%


Epoch 2/10:  91%|████████████████████████████████████████████████████▌     | 29/32 [52:03<05:17, 105.70s/it, loss=1.28]